# Predicting Bad Loans with Machine Learning - Part 2: Feature Engineering

### Content List
#### Imports
#### 

Feature Engineering Steps:
- Brainstorming or testing features.
- Deciding what features to create.
- Creating features.
- Checking how the features work with your model.
- Improving features (if needed).

### Imports
Similar imports as in Part 1

In [1]:

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

In [2]:
data_ = pd.read_csv('./cleaned_af.csv')

In [3]:
data = data_

### Check data types for columns
This is an important step as we need to have a clear idea of which how many object columns remain as we need to get them into an interpretable format. 

In [4]:
data.dtypes.value_counts()

float64    54
object     17
int64       2
dtype: int64

With 19 object columns present, we will need to consider multiple options for manipulating the data types. Most likely we will use a combination of ordinal rankings, one-hot dummy encoding, binarizing two-value features and taking the length of user submitted text columns.

In [5]:
object_cols = data.select_dtypes(include='object').columns
object_cols

Index(['term', 'grade', 'emp_title', 'emp_length', 'home_ownership',
       'verification_status', 'issue_d', 'pymnt_plan', 'desc', 'purpose',
       'title', 'addr_state', 'earliest_cr_line', 'application_type',
       'hardship_flag', 'disbursement_method', 'debt_settlement_flag'],
      dtype='object')

The following features will be binarized: term, verification_status, pyment_plan, application_type, hardship_flag, disbursement_method and debt_settlement_flag.

The following features will be one-hot encoded: home_ownership, purpose and addr_state.

The following features will be encoded with ordinal rank: emp_length and grade.

The text columns of desc, emp_title and title will each be measured by length, then combined into one feature for potential Natural Language Processing work.

The remaining text columns of issue_d and earliest_cr_line will be converted into DateTime format, then create a new column from their difference that will be converted into an integer.

In [6]:
data.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'pymnt_plan', 'desc',
       'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high',
       'last_fico_range_low', 'collections_12_mths_ex_med', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths',
       'delinq_amnt', 'mort_acc', 'num_accts_ever_120_pd', 'num_actv_bc_tl',
       'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl',
       'num_op_rev_tl', 'num_rev_accts', 'num_r

### User-Submitted Text Features

#### build an aggregate column of all text input

In [7]:
desc_length = []

[desc_length.append(len(i)) for i in data.desc]
    
data["desc_length"]= desc_length

In [8]:
data['desc_length'].head()

0    3
1    3
2    3
3    3
4    3
Name: desc_length, dtype: int64

In [9]:
data['all_text'] = data['desc'] + " " + data['emp_title'] + " " + data['title']

In [10]:
data['all_text'].head()

0                  .  leadman Debt consolidation
1                           .  Engineer Business
2                            .  truck driver  . 
3          .  Contract Specialist Major purchase
4     .  Veterinary Tecnician Debt consolidation
Name: all_text, dtype: object

#### Using List Comprehension, measure the lengths of each item dealing with user text input and return that as a new column

In [11]:
all_text_length = [len(i) for i in data['all_text']]
    
data["all_text_length"] = all_text_length

In [12]:
data['all_text_length'].value_counts()

38      60580
30      57492
39      56927
37      55074
35      51277
33      49551
40      46482
41      46009
36      44413
42      44295
43      41064
44      38316
32      37271
31      36667
34      35612
45      35335
28      35051
46      29577
47      28637
29      27050
26      23967
48      23175
49      19246
50      18970
25      16569
51      15563
27      14842
52      14439
53      11567
54      10114
        ...  
781         1
784         1
1664        1
788         1
790         1
792         1
798         1
799         1
801         1
803         1
1779        1
755         1
753         1
751         1
1714        1
3779        1
1747        1
730         1
737         1
739         1
741         1
742         1
743         1
744         1
745         1
746         1
747         1
748         1
750         1
1024        1
Name: all_text_length, Length: 973, dtype: int64

In [13]:
data.emp_title.head()

0                 leadman
1                Engineer
2            truck driver
3     Contract Specialist
4    Veterinary Tecnician
Name: emp_title, dtype: object

In [14]:
desc_length = [len(i) for i in data.desc]
    
data["desc_length"]= desc_length

In [15]:
emp_title_length = [len(i) for i in data.emp_title]

data["emp_title_length"] = emp_title_length

In [16]:
title_length = [len(i) for i in data.title]
    
data["title_length"] = title_length

### Ordinal Rankings: converting grade G:1 to A:7

In [17]:
#building a dictionary of values to identify and replace
length_dict = {'A':7,
        'B': 6,               
        'C': 5,
        'D': 4,
        'E': 3,
        'F': 2,
        'G': 1,}
data['grade'].replace(length_dict, inplace=True)

In [18]:
data['grade'].head()

0    5
1    5
2    6
3    2
4    5
Name: grade, dtype: int64

Next, employment length needs to be converted from strings to integers that represent the ordinal ranking of the values. First we need to convert less than 1 years to 0, then more than 10 years to 10. Although this is not a perfectly representative scale, it is good enough for our purposes.

In [19]:
#building a dictionary of values to identify and replace
length_dict = {'10+ years':10,
        '2 years': 2,
        '< 1 year': 0,
        '3 years': 3,
        '1 year': 1,
        '5 years': 5,
        '4 years': 4,
        '6 years': 6,
        '8 years': 8,
        '7 years': 7,
        '9 years': 9}
data['emp_length'].replace(length_dict, inplace=True)

In [20]:
data['emp_length'].value_counts()

10    428216
2     115916
3     102514
0     102306
1      84143
5      79155
4      75762
6      59046
8      58333
7      56600
9      48915
Name: emp_length, dtype: int64

VISUALIZE WITH SCATTERPLOT

In [21]:
data['emp_length'].value_counts(normalize=True)

10    0.353633
2     0.095727
3     0.084659
0     0.084487
1     0.069488
5     0.065368
4     0.062566
6     0.048762
8     0.048173
7     0.046742
9     0.040395
Name: emp_length, dtype: float64

In [22]:
# Find the mean employment length of people whose loans did or did not succeed.

data.groupby(by='classes')['emp_length'].mean()

classes
0    5.860851
1    6.033932
Name: emp_length, dtype: float64

### one hot encoding values for columns
- purpose
- verification_status
- addr_state
- pymnt_plan
- initial_list_status
- application_type
- hardship_flag
- disbursement_method
- debt_settlement_flag

In [23]:
## all remaining object cols
object_cols

Index(['term', 'grade', 'emp_title', 'emp_length', 'home_ownership',
       'verification_status', 'issue_d', 'pymnt_plan', 'desc', 'purpose',
       'title', 'addr_state', 'earliest_cr_line', 'application_type',
       'hardship_flag', 'disbursement_method', 'debt_settlement_flag'],
      dtype='object')

In [24]:
data['term'].head()

0     36 months
1     36 months
2     60 months
3     60 months
4     36 months
Name: term, dtype: object

#### Purpose

In [25]:
data['purpose'].value_counts()

debt_consolidation    710586
credit_card           270419
home_improvement       76917
other                  66739
major_purchase         25060
medical                13252
small_business         12448
car                    11737
moving                  8085
vacation                7788
house                   6281
wedding                  848
renewable_energy         746
Name: purpose, dtype: int64

In [26]:
data.groupby(by='classes')['purpose'].value_counts(normalize= True)

classes  purpose           
0        debt_consolidation    0.617561
         credit_card           0.188380
         other                 0.059608
         home_improvement      0.057523
         major_purchase        0.020580
         small_business        0.015711
         medical               0.011843
         moving                0.007896
         car                   0.007400
         vacation              0.006299
         house                 0.005974
         renewable_energy      0.000744
         wedding               0.000481
1        debt_consolidation    0.578500
         credit_card           0.232778
         home_improvement      0.065144
         other                 0.053899
         major_purchase        0.020726
         medical               0.010701
         car                   0.010313
         small_business        0.008810
         vacation              0.006467
         moving                0.006347
         house                 0.004974
         wed

In [27]:
# how to better visualize these representations?

In [28]:
one_hot = ['purpose', 'term',
'verification_status',
'pymnt_plan',
'application_type',
'hardship_flag', 'addr_state',
'disbursement_method', 
'debt_settlement_flag', 'home_ownership']

In [29]:
data = pd.get_dummies(data= data, columns = one_hot, drop_first=True)

In [30]:
data.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'grade', 'emp_title', 'emp_length', 'annual_inc',
       'issue_d',
       ...
       'addr_state_WI', 'addr_state_WV', 'addr_state_WY',
       'disbursement_method_DirectPay', 'debt_settlement_flag_Y',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT'],
      dtype='object', length=143)

# Preprocessing of Text Feature
- take the entire list of words in all_text and convert to 2 separate corpus based on whether they are associated with 1 or 0 for loan status

### TF-IDF on data['all_text']

### CountVectorizer word counter function

In [31]:
# # Function to count words in each df
# def word_counter(title_df,stop_list = []):
    
#     # Count Vectorize
#     cvec = CountVectorizer(stop_words = stop_list, max_features=15)

#     # Transform the corpus
#     X_text = cvec.fit_transform(title_df['Post Text'])

#     # Converts text to array form
#     X_text = pd.DataFrame(X_text.toarray(), columns= cvec.get_feature_names())

#     # See word counts
#     word_counts = X_text.sum().sort_values(0, ascending=False)
    
#     return word_counts

#### Interaction Columns

In [32]:
dataCorr = data.corr(method='pearson')
dataCorr = dataCorr[abs(dataCorr) >= 0.8].stack().reset_index()
dataCorr = dataCorr[dataCorr['level_0'].astype(str)!=dataCorr['level_1'].astype(str)]


In [33]:
# filtering out lower/upper triangular duplicates 
dataCorr['ordered-cols'] = dataCorr.apply(lambda x: '-'.join(sorted([x['level_0'],x['level_1']])),axis=1)
dataCorr = dataCorr.drop_duplicates(['ordered-cols'])
dataCorr.drop(['ordered-cols'], axis=1, inplace=True)

In [34]:
dataCorr = pd.DataFrame(dataCorr)

dataCorr.columns=['Initial', 'Secondary', 'Corr_Values']
#generate absolute value of corr values
dataCorr['Abs_Corr_Values'] = abs(dataCorr['Corr_Values'])
# initial_cols = list(dataCorr['Initial'])

# secondary_cols = list(dataCorr['Secondary'])

In [35]:
dataCorr = dataCorr.sort_values(by= 'Abs_Corr_Values', ascending = False)
dataCorr = dataCorr.iloc[6:]
dataCorr

In [36]:
for i,j in zip(dataCorr['Initial'], dataCorr['Secondary']):
    name = i + '*' + j  
    data[name] = data[i] * data[j]
    print(name)

desc_length*all_text_length
num_actv_rev_tl*num_rev_tl_bal_gt_0
tot_cur_bal*tot_hi_cred_lim
recoveries*collection_recovery_fee
int_rate*grade
funded_amnt*installment
loan_amnt*installment
funded_amnt_inv*installment
total_bal_ex_mort*total_il_high_credit_limit
pymnt_plan_y*hardship_flag_Y
revol_util*bc_util
bc_util*percent_bc_gt_75
tot_cur_bal*avg_cur_bal
num_bc_tl*num_rev_accts
last_fico_range_high*last_fico_range_low
open_acc*num_op_rev_tl
num_op_rev_tl*num_sats
num_actv_bc_tl*num_bc_sats
home_ownership_MORTGAGE*home_ownership_RENT
num_actv_bc_tl*num_actv_rev_tl
avg_cur_bal*tot_hi_cred_lim
num_actv_bc_tl*num_rev_tl_bal_gt_0


In [37]:
#all of the values in this column are 0 since they are non-overlapping binary values
data.drop(columns = ['home_ownership_MORTGAGE*home_ownership_RENT'], inplace= True)

In [38]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,emp_title,emp_length,annual_inc,issue_d,...,bc_util*percent_bc_gt_75,tot_cur_bal*avg_cur_bal,num_bc_tl*num_rev_accts,last_fico_range_high*last_fico_range_low,open_acc*num_op_rev_tl,num_op_rev_tl*num_sats,num_actv_bc_tl*num_bc_sats,num_actv_bc_tl*num_actv_rev_tl,avg_cur_bal*tot_hi_cred_lim,num_actv_bc_tl*num_rev_tl_bal_gt_0
0,3600.0,3600.0,3600.0,13.99,123.03,5,leadman,10,55000.0,2015-12-01,...,0.00,2.999658e+09,45.0,315840.0,28.0,28.0,4.0,8.0,3.685813e+09,8.0
1,24700.0,24700.0,24700.0,11.99,820.28,5,Engineer,10,65000.0,2015-12-01,...,208.67,1.989386e+09,459.0,485805.0,440.0,440.0,65.0,25.0,3.056327e+09,25.0
2,20000.0,20000.0,20000.0,10.78,432.66,6,truck driver,10,63000.0,2015-12-01,...,2795.00,5.997713e+09,28.0,492800.0,24.0,24.0,4.0,6.0,6.905722e+09,6.0
3,10400.0,10400.0,10400.0,22.45,289.91,2,Contract Specialist,3,104433.0,2015-12-01,...,4650.00,9.170344e+09,171.0,492800.0,84.0,84.0,20.0,24.0,1.215147e+10,24.0
4,11950.0,11950.0,11950.0,13.44,405.18,5,Veterinary Tecnician,4,34000.0,2015-12-01,...,9100.00,3.276288e+07,8.0,573045.0,20.0,20.0,4.0,6.0,4.326400e+07,6.0


In [39]:
data.shape

(1210906, 164)

In [40]:
data.corr()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,emp_length,annual_inc,dti,delinq_2yrs,...,bc_util*percent_bc_gt_75,tot_cur_bal*avg_cur_bal,num_bc_tl*num_rev_accts,last_fico_range_high*last_fico_range_low,open_acc*num_op_rev_tl,num_op_rev_tl*num_sats,num_actv_bc_tl*num_bc_sats,num_actv_bc_tl*num_actv_rev_tl,avg_cur_bal*tot_hi_cred_lim,num_actv_bc_tl*num_rev_tl_bal_gt_0
loan_amnt,1.000000,0.999999,0.999994,0.139270,0.952266,-0.138890,0.082787,0.302155,0.028150,-0.007105,...,0.014846,0.168176,0.153768,0.051510,0.151051,0.151353,0.175135,0.152968,0.166044,0.153694
funded_amnt,0.999999,1.000000,0.999994,0.139269,0.952267,-0.138888,0.082788,0.302154,0.028150,-0.007105,...,0.014847,0.168177,0.153767,0.051511,0.151050,0.151351,0.175135,0.152968,0.166044,0.153694
funded_amnt_inv,0.999994,0.999994,1.000000,0.139251,0.952221,-0.138861,0.082804,0.302183,0.028086,-0.007125,...,0.014803,0.168193,0.153755,0.051539,0.151043,0.151344,0.175125,0.152948,0.166061,0.153673
int_rate,0.139270,0.139269,0.139251,1.000000,0.151851,-0.954909,-0.008513,-0.075934,0.164399,0.043206,...,0.243202,-0.052347,-0.054052,-0.332678,-0.000277,-0.000506,-0.001029,0.052205,-0.056716,0.053368
installment,0.952266,0.952267,0.952221,0.151851,1.000000,-0.141801,0.070435,0.293448,0.030747,0.002150,...,0.033802,0.156970,0.143661,0.038526,0.145822,0.146060,0.174919,0.158183,0.154809,0.158914
grade,-0.138890,-0.138888,-0.138861,-0.954909,-0.141801,1.000000,0.008108,0.074031,-0.167070,-0.047904,...,-0.244775,0.051980,0.052573,0.334643,-0.004201,-0.003865,0.000488,-0.054029,0.056126,-0.055011
emp_length,0.082787,0.082788,0.082804,-0.008513,0.070435,0.008108,1.000000,0.062960,0.020588,0.019218,...,0.027609,0.041184,0.108443,0.025996,0.078450,0.078100,0.071720,0.092986,0.042661,0.094831
annual_inc,0.302155,0.302154,0.302183,-0.075934,0.293448,0.074031,0.062960,1.000000,-0.158502,0.037949,...,-0.015069,0.310131,0.100856,0.065746,0.093234,0.093166,0.105052,0.087468,0.302166,0.087221
dti,0.028150,0.028150,0.028086,0.164399,0.030747,-0.167070,0.020588,-0.158502,1.000000,-0.013396,...,0.140474,-0.068473,0.070976,-0.072081,0.172828,0.172877,0.115816,0.162659,-0.063041,0.166127
delinq_2yrs,-0.007105,-0.007105,-0.007125,0.043206,0.002150,-0.047904,0.019218,0.037949,-0.013396,1.000000,...,-0.014558,0.026477,0.042509,-0.061565,0.017066,0.016073,-0.029869,-0.022593,0.026482,-0.025418


In [41]:
data.drop(columns= ['funded_amnt', 'funded_amnt_inv'], inplace=True)

## With the addition of our engineered columns, the Feature Engineering stage is complete. Next, exporting the polished dataframe as a csv in order to get it into the third and final notebook for Modeling.

In [42]:
data.to_csv('./cleaned_FEATURES.csv', index=False)